In [12]:
import pickle
import os
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix, csr_matrix

from lightfm import LightFM

In [5]:
os.getcwd()

'C:\\Users\\SSAFY\\Desktop\\Whisky\\v1'

In [25]:
## fastAPI 에서는 mysql db에서 조회해온다.
whisky = pd.read_csv("../dataset/whisky.csv", index_col=0 , encoding="UTF-8")
rating = pd.read_csv("../dataset/rating.csv", index_col=0, encoding="UTF-8")

In [2]:
# Load the saved model
with open('origin_user_rating_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [48]:
def sample_recommendation(model, data, user_id, item_features, cost_rank, k = 9):
    

    n_items = item_features.shape[0]
    
    scores = model.predict(user_ids=user_id, item_ids=np.arange(n_items), item_features=item_features)
#     top_items = data[]
    print(np.argsort(-scores))
    return np.argsort(-scores)
#     for user_id in user_ids:
#         known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
#         scores = model.predict(user_id, np.arange(n_items))
#         top_items = data['item_labels'][np.argsort(-scores)]
        
#         print("User %s" % user_id)
#         print("     Known positives:")
        
#         for x in known_positives[:3]:
#             print("        %s" % x)

#         print("     Recommended:")
        
#         for x in top_items[:3]:
#             print("        %s" % x)
        
# sample_recommendation(model, data, [3, 25, 450]) 

In [18]:
def get_item_features(data):
    columns = data.columns.tolist()
    columns = [columns[8]] + columns[11:]
    item_features = np.zeros((data['whisky_id'].nunique(), len(columns)))
    for i in range(data['whisky_id'].nunique()):
        for j, col_name in enumerate(columns):
            item_features[i, j] = data.iloc[i][col_name]
    return csr_matrix(item_features)
    # item_features = csr_matrix(item_features)

In [19]:
item_features = get_item_features(whisky)

In [21]:
print(item_features)

  (0, 0)	5.0
  (0, 1)	30.0
  (0, 2)	85.0
  (0, 3)	50.0
  (0, 4)	30.0
  (0, 5)	20.0
  (0, 6)	80.0
  (0, 7)	80.0
  (0, 8)	85.0
  (0, 9)	15.0
  (0, 10)	20.0
  (0, 11)	25.0
  (0, 12)	85.0
  (0, 13)	50.0
  (1, 0)	4.0
  (1, 1)	40.0
  (1, 2)	30.0
  (1, 3)	40.0
  (1, 4)	20.0
  (1, 5)	40.0
  (1, 6)	70.0
  (1, 7)	80.0
  (1, 8)	70.0
  (1, 9)	40.0
  (1, 10)	50.0
  :	:
  (3532, 4)	10.0
  (3532, 5)	40.0
  (3532, 6)	10.0
  (3532, 7)	10.0
  (3532, 8)	90.0
  (3532, 10)	50.0
  (3532, 11)	20.0
  (3532, 12)	30.0
  (3532, 13)	10.0
  (3533, 0)	2.0
  (3533, 3)	10.0
  (3533, 4)	100.0
  (3533, 5)	80.0
  (3533, 8)	40.0
  (3533, 10)	20.0
  (3534, 0)	1.0
  (3534, 1)	31.0
  (3534, 3)	40.0
  (3534, 4)	29.0
  (3534, 6)	30.0
  (3534, 7)	40.0
  (3534, 8)	100.0
  (3534, 10)	55.0
  (3534, 12)	30.0
  (3534, 13)	10.0


In [62]:
rec = sample_recommendation(model, whisky, 0, item_features, 3)[:20]

[3521  669  810 ... 3190 3533 3487]


In [69]:
my_rating = rating.loc[rating["user_id"] == 0].sort_values(by='rating', ascending=False)["whisky_id"]

In [74]:
rating.loc[rating["user_id"] == 0].sort_values(by="rating", ascending=False)

,user_id,whisky_id,rating
0,0,0,9.0
20895,0,811,9.0
30094,0,546,8.0
53435,0,1370,8.0
2461,0,1769,8.0
7343,0,1994,6.0
7295,0,2451,3.0


In [71]:
whisky.iloc[my_rating][ ["name","category", "abv", "total_rating", whisky.columns.tolist()[8]] + whisky.columns.tolist()[11:] ]

,name,category,abv,total_rating,cost_rank,smoky,peaty,spicy,herbal,oily,full_bodied,rich,sweet,salty,vanilla,tart,fruity,floral
0,Hibiki 21 Year,Blended,43.0,861,5,30,85,50,30,20,80,80,85,15,20,25,85,50
811,Yamazaki 12 Year,Single Malt,43.0,2859,4,20,30,40,20,10,60,50,70,20,70,60,70,50
546,Yamazaki Distiller's Reserve,Peated Single Malt,43.0,396,3,10,0,60,40,55,70,65,80,0,80,70,95,75
1370,Hibiki 12 Year,Blended,43.0,1070,3,30,10,40,40,30,60,70,70,20,70,60,70,80
1769,Suntory AO World Whisky,Other Whiskey,43.0,63,2,20,10,55,15,25,54,54,65,5,15,35,45,0
1994,Maker's Mark Bourbon,Bourbon,45.0,8675,2,10,0,40,30,10,50,70,80,0,90,20,10,0
2451,2 Gingers,Blended,40.0,327,1,20,0,40,30,40,70,70,70,0,60,80,70,10


In [72]:
whisky.iloc[rec][ ["name","category", "abv", "total_rating", whisky.columns.tolist()[8]] + whisky.columns.tolist()[11:] ]

,name,category,abv,total_rating,cost_rank,smoky,peaty,spicy,herbal,oily,full_bodied,rich,sweet,salty,vanilla,tart,fruity,floral
3521,Teton Moonshine Spiced Apple Pie Whiskey,Flavored Whiskey,35.0,2,1,0,0,40,15,10,45,65,85,0,100,25,100,50
669,Daviess County Kentucky Straight Bourbon Frenc...,Bourbon,48.0,119,2,10,0,60,0,60,80,75,90,0,90,30,80,95
810,Jack Daniel's Gentleman Jack,Tennessee,40.0,3919,2,30,0,20,0,50,50,60,80,30,70,10,90,20
2382,Nomad Outland Blended Whisky,Blended,41.3,124,2,0,0,50,0,50,70,90,90,0,70,60,80,70
374,Benromach Peat Smoke 2005,Peated Single Malt,46.0,104,3,80,70,40,10,30,80,80,80,50,60,0,80,10
3471,Tirado Gold Corn Whiskey,Corn,40.0,3,2,0,0,10,0,20,85,85,90,0,80,5,50,0
3393,Blaum Bros Knotter Bourbon 3 Year (Batch #6),Bourbon,45.0,17,2,10,0,65,0,30,85,75,80,0,35,10,85,35
785,George Dickel Barrel Select,Tennessee,43.0,691,2,40,0,70,30,50,80,85,80,10,75,0,80,65
1180,Daviess County Kentucky Straight Bourbon,Bourbon,48.0,85,2,45,0,65,20,70,80,85,85,0,85,60,70,30
3453,Duggan's Dew O'Kirkintilloch,Blended,43.4,22,1,10,0,5,0,10,60,60,90,0,70,0,50,10


In [ ]:
model.